In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataloader
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import display
import torch.optim as optim

In [30]:
device = torch.device('cuda')

### Load Data

In [31]:
# Uncomment the below line if you haven't created the dataset file yet
#!python prepare_training_data.py 

# Load in the data file
df = pd.read_pickle('../data/training_set.pkl')
display(df)

class Data(Dataset):
    def __init__(self, x, y):
        super(Data, self).__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

train_slices = df['slice'].to_numpy()
data = Data(train_slices,train_slices)

,slice_name,slice
0,file_brain_AXT2_200_2000535_0,"[0.0029112822, 0.0022742343, 0.0024876709, 0.0..."
1,file_brain_AXT2_200_2000535_1,"[0.00254231, 0.0018121544, 0.0022545222, 0.002..."
2,file_brain_AXT2_200_2000535_2,"[0.0021948142, 0.001427773, 0.0019365047, 0.00..."
3,file_brain_AXT2_200_2000535_3,"[0.0019062636, 0.0010817663, 0.0016660133, 0.0..."
4,file_brain_AXT2_200_2000535_4,"[0.001538842, 0.00081564067, 0.0013866577, 0.0..."
...,...,...
425,file_brain_AXT2_200_2000368_11,"[5.3744112e-05, 5.3206768e-05, 5.620142e-05, 5..."
426,file_brain_AXT2_200_2000368_12,"[5.141851e-05, 5.086812e-05, 5.1086612e-05, 5...."
427,file_brain_AXT2_200_2000368_13,"[5.5321667e-05, 5.2995976e-05, 5.153848e-05, 5..."
428,file_brain_AXT2_200_2000368_14,"[5.176266e-05, 5.110678e-05, 5.0593735e-05, 5...."


### Define the Neural Network

In [32]:
# Inherit from torch.nn, define basic k-latent AutoEncoder
k_latent = 2500

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.k_latent_encode = nn.Linear(256*256, k_latent)
        self.k_latent_decode = nn.Linear(k_latent, 256*256)

    def forward(self, x):
        x = self.k_latent_encode(x)
        x = torch.relu(x)
        x = self.k_latent_decode(x)
        x = torch.sigmoid(x)
        return x

### Train the Model

In [33]:
# Training setup
n_epochs = 100
minibatch_size = 16
batch_loss = np.zeros(n_epochs)

model = AutoEncoder()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [27]:
# Do training
for epoch in range(n_epochs):
    # Pick batches for this epoch
    train_loader = torch.utils.data.DataLoader(data, batch_size=minibatch_size, shuffle=True)
    for batch_idx, (X, _) in enumerate(train_loader):
        optimizer.zero_grad()   # zero the gradient buffers
        Y = model(X)
        loss = criterion(X,Y)
        batch_loss[epoch]+=loss.item() # save off the loss
        loss.backward()
        optimizer.step()    # update
    print("Completed epoch: "+str(epoch)+" , Loss: "+str(batch_loss[epoch]))

Completed epoch: 0 , Loss: [13.49960357  0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
Completed epoch: 1 , Loss: [13.49960357 13.49957031  0.          0.          0.          0.
  0.          0.          0.          0.        ]
Completed epoch: 2 , Loss: [13.49960357 13.49957031 13.49953763  0.          0.          0.
  0.          0.          0.          0.        ]


KeyboardInterrupt: 